# Hyper-parameter Optimization Results

To effectively decide which parameters to use on our models, we have to evaluate the HPO results on the GoEmotions dataset.

In [1]:
import glob
import json
import pandas as pd

results = []
paths = glob.glob('../output/*/*/*.json', recursive=True)
for path in paths:
    with open(path) as fp:
        result_dict = json.load(fp)
        result_tuple = (path, result_dict)
        results.append(result_tuple)

results_df = pd.DataFrame(results, columns=['Path', 'Dict'])

In [2]:
EXPERIMENT_METRICS = ['macro_f1', 'micro_f1']
EXPERIMENT_COLUMNS = ['Dataset', 'ModelType', 'Experiment']


def parse_path(path):
    dataset, model_type, experiment = path.split('/')[-3:]
    experiment = experiment.split('.')[0]
    return dataset, model_type, experiment


def build_experiment_columns(dataframe):
    path_parts = [parse_path(path) for path in dataframe.Path]
    for i, col in enumerate(EXPERIMENT_COLUMNS):
        dataframe[col] = [parts[i] for parts in path_parts]
    return dataframe


def extract_macro_stats(dataframe, metrics=EXPERIMENT_METRICS):
    split_names = dataframe['Dict'][0]['config']['data_config']['split_names']
    for split in split_names:
        for metric in metrics:
            new_col_name = '{}_{}'.format(split, metric)
            dataframe[new_col_name] = [exp_dict['results'][split][metric] for exp_dict in dataframe['Dict']]
    return dataframe


def extract_model_details(dataframe):
    dataframe['Extractor'] = [exp_dict['config']['extractor_config']['ex_type'] for exp_dict in dataframe['Dict']]
    dataframe['Model'] = [exp_dict['config']['model_config']['model_name'] for exp_dict in dataframe['Dict']]
    return dataframe


def parse_df(dataframe):
    dataframe = build_experiment_columns(dataframe)
    dataframe = extract_model_details(dataframe)
    dataframe = extract_macro_stats(dataframe)
    return dataframe


In [3]:
parsed_df = parse_df(results_df)

parsed_df.sort_values('valid_macro_f1')

,Path,Dict,Dataset,ModelType,Experiment,Extractor,Model,train_macro_f1,train_micro_f1,valid_macro_f1,valid_micro_f1,test_macro_f1,test_micro_f1
78,../output/GoEmotions/classic/98f6bca9a4ab65148...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,98f6bca9a4ab651482d7e1b22f78f86b,tfidf,sgd,0.086467,0.237511,0.086287,0.237473,0.087133,0.243009
26,../output/GoEmotions/classic/a93108cca090c2dfb...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,a93108cca090c2dfbbbc383d76859d23,tfidf,sgd,0.087561,0.239064,0.087443,0.238925,0.088382,0.244441
71,../output/GoEmotions/classic/bc505ed7a45b78401...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,bc505ed7a45b78401005670115de7804,tfidf,sgd,0.087547,0.219247,0.087490,0.218868,0.089887,0.224112
54,../output/GoEmotions/classic/6fe54fe68812f68c6...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,6fe54fe68812f68c66347614314b3d77,tfidf,sgd,0.087060,0.217988,0.087628,0.218700,0.088405,0.223826
61,../output/GoEmotions/classic/4c756098a05e6be85...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,4c756098a05e6be8553407e62531326e,tfidf,sgd,0.087445,0.218576,0.088262,0.219786,0.088820,0.224594
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,../output/GoEmotions/neural/271fd617e11526e0b1...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,271fd617e11526e0b11c722106684499,bert,dnnpool,0.558377,0.598313,0.511532,0.558837,0.508289,0.555420
931,../output/GoEmotions/neural/166f622070a0b55d14...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,166f622070a0b55d144984bc6ac61d90,bert,dnnpool,0.573776,0.608303,0.511793,0.556911,0.512084,0.554972
389,../output/GoEmotions/neural/19f0c349f25ed12523...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,19f0c349f25ed125238e9c08c60c55da,bert,dnnpool,0.575695,0.609094,0.512066,0.556384,0.506743,0.554735
752,../output/GoEmotions/neural/066bf75e0a90f449f5...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,066bf75e0a90f449f51743fda65e4f02,bert,dnnpool,0.564646,0.613024,0.512732,0.562093,0.496617,0.557918


In [8]:
keyed_groups = parsed_df.groupby(['Dataset', 'ModelType', 'Extractor', 'Model'])
best_valid_df = keyed_groups.valid_macro_f1.max().to_frame().reset_index()

parsed_df.merge(best_valid_df, on=['Dataset', 'ModelType', 'Extractor', 'Model', 'valid_macro_f1'], how='inner')

,Path,Dict,Dataset,ModelType,Experiment,Extractor,Model,train_macro_f1,train_micro_f1,valid_macro_f1,valid_micro_f1,test_macro_f1,test_micro_f1
0,../output/GoEmotions/classic/861e1116d39410758...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,861e1116d3941075823bf7224a5cf1c4,tfidf,naivebayes,0.613207,0.625921,0.347082,0.431262,0.333460,0.435667
1,../output/GoEmotions/classic/59de23e2d521b4565...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,59de23e2d521b45651c3b53a887ac80e,tfidf,sgd,0.627998,0.671968,0.478281,0.530271,0.471252,0.528617
2,../output/GoEmotions/neural/151678859634934c70...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,151678859634934c709006ec9f404611,bert,lstm,0.538513,0.607887,0.499987,0.572144,0.489639,0.570233
3,../output/GoEmotions/neural/0ae5faa3b9dd41390d...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,0ae5faa3b9dd41390d57d7e006254968,bert,dnnpool,0.593322,0.634571,0.515087,0.559888,0.499480,0.556102
